# TensorFlow Assignment: Multilayer Perceptron (MLP) and Convolutional Neural Network (CNN)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: [Fanhong Li]

Now that you've run through a simple logistic regression model on MNIST, let's see if we can do better (Hint: we can). For this assignment, you'll build a multilayer perceptron (MLP) and a convolutional neural network (CNN), two popular types of neural networks, and compare their performance. Some potentially useful code:

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Import data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# Helper functions for creating weight variables
def weight_variable(shape):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Tensorflow Functions that might also be of interest:
# tf.nn.sigmoid()
# tf.nn.relu()

### Multilayer Perceptron

Build a multilayer perceptron for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> fully connected (500 hidden units) -> nonlinearity (Sigmoid/ReLU) -> fully connected (10 hidden units) -> softmax

Skeleton framework for you to fill in (Code you need to provide is marked by `###`):

In [4]:
# Model Inputs
###x is every picture and y_ is label to each picture
x = tf.placeholder(tf.float32, [None, 784])### MNIST images enter graph here ###
y_ = tf.placeholder(tf.float32,[None, 10])### MNIST labels enter graph here ###

# Define the graph


### Create your MLP here#
W1 = weight_variable([784,500])
b1 = bias_variable([500])
W2 = weight_variable([500,10])
b2 = bias_variable([10])
#W3 = weight_variable([100,10])
#b3 = bias_variable([10])
h1 = tf.nn.sigmoid(tf.matmul(x, W1) + b1)
y_mlp = tf.matmul(h1, W2) + b2
#h2 = tf.nn.relu(tf.matmul(h1, W2) + b2)
#y_mlp = tf.matmul(h2, W3) + b3
### Make sure to name your MLP output as y_mlp ###


# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_mlp))

# Optimizer
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_mlp, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Training regimen
    for i in range(4000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(100)
                validation_accuracy += (1/10) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))
        
        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

step 0, validation accuracy 0.095
step 250, validation accuracy 0.905
step 500, validation accuracy 0.913
step 750, validation accuracy 0.924
step 1000, validation accuracy 0.928
step 1250, validation accuracy 0.943
step 1500, validation accuracy 0.937
step 1750, validation accuracy 0.947
step 2000, validation accuracy 0.957
step 2250, validation accuracy 0.955
step 2500, validation accuracy 0.945
step 2750, validation accuracy 0.96
step 3000, validation accuracy 0.963
step 3250, validation accuracy 0.974
step 3500, validation accuracy 0.958
step 3750, validation accuracy 0.965
test accuracy 0.9653


#### Comparison

How do the sigmoid and rectified linear unit (ReLU) compare?

***

sigmoid test accuracy is 0.9642
ReLU test accuracy is 0.9759

Out of curiousity I tried to implement ReLU twice, or both ReLU and sigmoid, with 2 hidden layers (500,100,10 units) and the results didn't change much:

double sigmoid: test accuracy 0.9584
double ReLU: test accuracy 0.9764
ReLU-sigmoid: test accuracy 0.979
sigmoid-ReLU: test accuracy 0.9495 and the validation accuracy is under 0.1 in step 2000 and rised rapidly after. I restarted the notebook and the test accuracy is 0.1135 which is strange, so i tried again and this time it stays around 0.1 in step 500 and rised up to 0.9637 in the end. I think trough these tests first use sigmoid then ReLU might make the result unstable.

Also I've tried different unit size, for 600 units, the sigmoid test accuracy 0.9634; For 100 units, the speed is considerably faster yet with sigmoid test accuracy 0.9616, a bit less. And for 20 units test accuracy is 0.9466. 10 units is 0.9141, since for logistic regression the accuracy's about 0.92, it's even worsening the results, while ReLU is making it a bit better to about 0.9254. Also, when there's 984 units in the hidden layer, the accuracy goes up to 0.978 for ReLU and 0.9645 for sigmoid.

Overall, ReLU has a better accuracy and the less units a hidden layer have, the faster the training process will be and the less accuracy the output will be.

***

### Convolutional Neural Network

Build a simple 2-layer CNN for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> CNN (32 5x5 filters) -> nonlinearity (ReLU) ->  (2x2 max pool) -> CNN (64 5x5 filters) -> nonlinearity (ReLU) -> (2x2 max pool) -> fully connected (1024 hidden units) -> nonlinearity (ReLU) -> fully connected (10 hidden units) -> softmax

Some additional functions that you might find helpful:

In [5]:
# Convolutional neural network functions
def conv2d(x, W):
    """conv2d returns a 2d convolution layer with full stride."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    """max_pool_2x2 downsamples a feature map by 2X."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Tensorflow Function that might also be of interest:
# tf.reshape()

Skeleton framework for you to fill in (Code you need to provide is marked by `###`):

*Hint: Convolutional Neural Networks are spatial models. You'll want to transform the flattened MNIST vectors into images for the CNN. Similarly, you might want to flatten it again sometime before you do a softmax. You also might want to look into the  [conv2d() documentation](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d) to see what shape kernel/filter it's expecting.*

In [10]:
# Model Inputs
x = tf.placeholder(tf.float32, [None, 784])### MNIST images enter graph here ###
y_ = tf.placeholder(tf.float32,[None, 10])### MNIST labels enter graph here ###

# Define the graph
x_image = tf.reshape(x, [-1,28,28,1])

### Create your CNN here##
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2
### Make sure to name your CNN output as y_conv ###

# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

# Optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Training regimen
    for i in range(10000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(50)
                validation_accuracy += (1/10) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))
        
        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

step 0, validation accuracy 0.1
step 250, validation accuracy 0.906
step 500, validation accuracy 0.936
step 750, validation accuracy 0.948
step 1000, validation accuracy 0.964
step 1250, validation accuracy 0.966
step 1500, validation accuracy 0.974
step 1750, validation accuracy 0.98
step 2000, validation accuracy 0.964
step 2250, validation accuracy 0.986
step 2500, validation accuracy 0.98
step 2750, validation accuracy 0.978
step 3000, validation accuracy 0.984
step 3250, validation accuracy 0.982
step 3500, validation accuracy 0.982
step 3750, validation accuracy 0.994
step 4000, validation accuracy 0.984
step 4250, validation accuracy 0.992
step 4500, validation accuracy 0.978
step 4750, validation accuracy 0.99
step 5000, validation accuracy 0.99
step 5250, validation accuracy 0.984
step 5500, validation accuracy 0.984
step 5750, validation accuracy 0.99
step 6000, validation accuracy 0.982
step 6250, validation accuracy 0.982
step 6500, validation accuracy 0.986
step 6750, val

Some differences from the logistic regression model to note:

- The CNN model might take a while to train. Depending on your machine, you might expect this to take up to half an hour. If you see your validation performance start to plateau, you can kill the training.

- The logistic regression model we used previously was pretty basic, and as such, we were able to get away with using the GradientDescentOptimizer, which performs implements the gradient descent algorithm. For more difficult optimization spaces (such as the ones deep networks pose), we might want to use more sophisticated algorithms. Prof David Carlson has a lecture on this later.
    
- Because of the larger size of our network, notice that our minibatch size has shrunk.
    
- We've added a validation step every 250 minibatches. This let's us see how our model is doing during the training process, rather than sit around twiddling our thumbs and hoping for the best when training finishes. This becomes especially significant as training regimens start approaching days and weeks in length. Normally, we validate on the entire validation set, but for the sake of time we'll just stick to 10 validation minibatches (500 images) for this homework assignment.

#### Comparison

How do the MLP and CNN compare in accuracy? Training time? Why would you use one vs the other? Is there a problem you see with MLPs when applied to other image datasets?

***

my MLP with RaLU is with an accuracy of 0.976 with an error rate of 0.024 and CNN's accuracy is 0.988 with an error rate of 0.012,
CNN have cut the error rate in half. Yet the training time is nearly 100 times of MLP. 

I would use CNN in this case since this one-time training can ensure a much less error rate and the time is still tolerable once the training is done it can be used over again. In other cases when accuracy is not so necesary or the improve is not evident and the training time is too long I would choose the shorter and less accurate one.

No, because in this case, we are simply training small image with only 1 color so the veriable number is limited to 784, if there's multiple color channel with larger photo, such as 1024 * 1024 * 3 there would be 3,145,728 variable and it's hard to process. In such case the MLP might be impossible to apply.

***